In [1]:
import matplotlib
matplotlib.use("nbagg")

import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

from utility import *
from models import *

In [2]:
import math

import torch
import torch.nn as nn
import torch.utils as utils
import torch.nn.init as init
import torch.utils.data as data
import torchvision.utils as v_utils
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import Tensor

import torch.optim as optim
import torch.nn.functional as F

In [3]:
from os import listdir
from os.path import isfile, join
import numpy as np

polycube_path = "/Users/davidcleres/DeepShape/Polycubing-Automated/Generated-Cars/"
polycube_files = [f for f in listdir(polycube_path) if isfile(join(polycube_path, f))]

voxelized_mesh_path = "/Users/davidcleres/DeepShape/Polycubing-Automated/voxelizedMeshes/"
voxelized_mesh_files = [f for f in listdir(voxelized_mesh_path) if isfile(join(voxelized_mesh_path, f))]

voxelizedFiles = []
polycubedFiles = []

for f in voxelized_mesh_files: 
    if f[-13:] == "voxelized.txt":
        voxelizedFiles = np.hstack((voxelizedFiles, f))
    
for f in polycube_files:
    if f[-14:] == "finalCubes.txt":
        polycubedFiles = np.hstack((polycubedFiles, f))

# Define the global parameters 

In [4]:
grid_size=32
batch_size=15

### Save the tensor to a text file 

In [5]:
voxelized_train_input, polycube_target=loadData(grid_size, polycube_path, voxelized_mesh_path, voxelizedFiles, polycubedFiles, loadFromScratch=True)

<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch

# Load Data

In [6]:
preprocessed_input_train, preprocessed_input_validation, preprocessed_input_train_target, preprocessed_input_validation_target = preprocessing_train(voxelized_train_input, polycube_target,batch_size, False, False)

preprocessed_input_train = torch.from_numpy(preprocessed_input_train)
preprocessed_input_validation = torch.from_numpy(preprocessed_input_validation)
preprocessed_input_train_target = torch.from_numpy(preprocessed_input_train_target)
preprocessed_input_validation_target = torch.from_numpy(preprocessed_input_validation_target)

Ntrain = len(preprocessed_input_train[:, 0,0,0,0]) 
Nvalidation = len(preprocessed_input_validation[:,0,0,0,0])

train_input = Variable(preprocessed_input_train.view(Ntrain, 1, grid_size, grid_size, grid_size).float())
validation_input = Variable(preprocessed_input_validation.view(Nvalidation, 1, grid_size, grid_size, grid_size).float())

labels_train = preprocessed_input_train_target.float()
labels_validation = preprocessed_input_validation_target.float()

print('train', train_input.shape)
print('validation', validation_input.shape)
print('train_target', labels_train.shape)
print('validation_target', labels_validation.shape)

train torch.Size([45, 1, 32, 32, 32])
validation torch.Size([15, 1, 32, 32, 32])
train_target torch.Size([45, 32, 32, 32])
validation_target torch.Size([15, 32, 32, 32])


# Network initialization

In [ ]:
# Train network 
#criterion = nn.BCELoss()
criterion = nn.CrossEntropyLoss()
#criterion = nn.PoissonNLLLoss()
#criterion = nn.BCEWithLogitsLoss()
#criterion = nn.SmoothL1Loss() #interesting ... but does not converge
#criterion = nn.MSELoss() #0.83 but unstable

if isinstance(criterion, nn.CrossEntropyLoss):
    train_target = Variable(preprocessed_input_train_target).long()  # keep long tensors
    validation_target = Variable(preprocessed_input_validation_target, volatile=True).long() # convert to float
    Noutputs = 2
    
elif isinstance(criterion, nn.NLLLoss):
    train_target = Variable(preprocessed_input_train_target)  # keep long tensors
    validation_target = Variable(preprocessed_input_validation_target, volatile=True) # convert to float
    Noutputs = 2
    
else:
    train_target = Variable(preprocessed_input_train_target.float()) # convert to float
    validation_target = Variable(preprocessed_input_validation_target.float(), volatile=True ) # convert to float
    Noutputs = 1
    
Nbatches = int(math.ceil(Ntrain/batch_size)) #batch_size is defined above
Nepochs = 1000
Nrep = 1
        
#model = conv3DNet(grid_size, Noutputs, batch_size)
#model = conv3DNet(grid_size, Noutputs, batch_size)
#model = conv3DNet(grid_size, Noutputs, batch_size)
#model = conv3DNet(grid_size, Noutputs, batch_size)
model = UnetGenerator_3d(in_dim=1, out_dim=Noutputs, num_filter=4)

#optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.90)
optimizer = optim.Adam(model.parameters())
#optimizer = optim.Adagrad(model.parameters())
#optimizer = optim.Adamax(model.parameters())
#optimizer = optim.ASGD(model.parameters())
#optimizer = optim.RMSprop(model.parameters())
#optimizer = optim.Rprop(model.parameters())
 
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, verbose=True) #Reduces the learning rate if it did not decreased by more than 10^-4 in 10 steps

train_errors = torch.Tensor(Nepochs).zero_()
validation_errors = torch.Tensor(Nepochs).zero_()

ep_loss = torch.Tensor(Nepochs).zero_()

for i_ep in range(Nepochs):
    for b_start in range(0, Ntrain, batch_size):
        bsize_eff = batch_size - max(0, b_start+batch_size-Ntrain)  # boundary case
        model.train()
        model.zero_grad()
        output = model(train_input.narrow(0, b_start, bsize_eff))
        if isinstance(criterion, nn.CrossEntropyLoss) or isinstance(criterion, nn.NLLLoss):
            batch_loss = criterion(output, train_target.narrow(0, b_start, bsize_eff))
        else:
            #if delta model is chosen
            #batch_loss = criterion(output.view(bsize_eff*Noutputs), train_target.narrow(0, b_start, bsize_eff))
            batch_loss = criterion(output.view(bsize_eff,grid_size,grid_size,grid_size), train_target.narrow(0, b_start, bsize_eff))
        ep_loss[i_ep] += batch_loss.data[0]
        batch_loss.backward()
        optimizer.step()

    scheduler.step(ep_loss[i_ep])

    nb_train_errs = compute_nb_errors(model, train_input, train_target, batch_size, criterion)
    nb_validation_errs = compute_nb_errors(model, validation_input, validation_target, batch_size, criterion)

    Ntrain_nb = Ntrain*grid_size**3
    Nvalidation_nb = Nvalidation*grid_size**3
    print("Epoch Number : ", i_ep)
    print("\t Training accuracy: ", (100*(Ntrain_nb-nb_train_errs)/Ntrain_nb))
    print("\t Validation accuracy ",(100*(Nvalidation_nb-nb_validation_errs)/Nvalidation_nb))

    print("\t Epoch Loss ", ep_loss[i_ep])

    train_errors[i_ep] = nb_train_errs
    validation_errors[i_ep] = nb_validation_errs


------Initiating U-Net------

Epoch Number :  0
	 Training accuracy:  45.57691786024306
	 Validation accuracy  45.007731119791664
	 Epoch Loss  2.1012232303619385
Epoch Number :  1
	 Training accuracy:  87.76075575086806
	 Validation accuracy  87.63936360677083
	 Epoch Loss  2.0755698680877686
Epoch Number :  2
	 Training accuracy:  90.39421929253473
	 Validation accuracy  90.0347900390625
	 Epoch Loss  2.060469388961792
Epoch Number :  3
	 Training accuracy:  92.5799560546875
	 Validation accuracy  92.45463053385417
	 Epoch Loss  2.0567660331726074
Epoch Number :  4
	 Training accuracy:  94.52663845486111
	 Validation accuracy  94.5068359375
	 Epoch Loss  2.054701805114746
Epoch Number :  5
	 Training accuracy:  93.34615071614583
	 Validation accuracy  93.32417805989583
	 Epoch Loss  2.0538744926452637
Epoch Number :  6
	 Training accuracy:  91.427001953125
	 Validation accuracy  91.14990234375
	 Epoch Loss  2.0530824661254883
Epoch Number :  7
	 Training accuracy:  90.63503689236111

Epoch Number :  63
	 Training accuracy:  94.48933919270833
	 Validation accuracy  93.41634114583333
	 Epoch Loss  1.9600108861923218
Epoch Number :  64
	 Training accuracy:  94.98942057291667
	 Validation accuracy  94.02079264322917
	 Epoch Loss  1.9588955640792847
Epoch Number :  65
	 Training accuracy:  94.635009765625
	 Validation accuracy  93.85579427083333
	 Epoch Loss  1.957463026046753
Epoch Number :  66
	 Training accuracy:  94.89854600694444
	 Validation accuracy  93.78377278645833
	 Epoch Loss  1.9576878547668457
Epoch Number :  67
	 Training accuracy:  95.27553982204861
	 Validation accuracy  94.57661946614583
	 Epoch Loss  1.954479455947876
Epoch Number :  68
	 Training accuracy:  94.85460069444444
	 Validation accuracy  93.81429036458333
	 Epoch Loss  1.9535503387451172
Epoch Number :  69
	 Training accuracy:  94.32990180121527
	 Validation accuracy  93.44095865885417
	 Epoch Loss  1.9498695135116577
Epoch Number :  70
	 Training accuracy:  95.17774793836806
	 Validation a

Epoch Number :  125
	 Training accuracy:  97.47097439236111
	 Validation accuracy  96.64896647135417
	 Epoch Loss  1.870856761932373
Epoch Number :  126
	 Training accuracy:  97.01605902777777
	 Validation accuracy  96.07625325520833
	 Epoch Loss  1.8685818910598755
Epoch Number :  127
	 Training accuracy:  97.72793240017361
	 Validation accuracy  96.942138671875
	 Epoch Loss  1.8669607639312744
Epoch Number :  128
	 Training accuracy:  96.96031358506944
	 Validation accuracy  95.7891845703125
	 Epoch Loss  1.8653641939163208
Epoch Number :  129
	 Training accuracy:  98.11733669704861
	 Validation accuracy  97.11710611979167
	 Epoch Loss  1.8635495901107788
Epoch Number :  130
	 Training accuracy:  97.11581759982639
	 Validation accuracy  95.8770751953125
	 Epoch Loss  1.8613219261169434
Epoch Number :  131
	 Training accuracy:  97.16396755642361
	 Validation accuracy  96.71366373697917
	 Epoch Loss  1.8594027757644653
Epoch Number :  132
	 Training accuracy:  97.83949110243056
	 Valid

Epoch Number :  187
	 Training accuracy:  99.73944769965277
	 Validation accuracy  97.901611328125
	 Epoch Loss  1.7930423021316528
Epoch Number :  188
	 Training accuracy:  96.43385145399306
	 Validation accuracy  96.036376953125
	 Epoch Loss  1.7911791801452637
Epoch Number :  189
	 Training accuracy:  99.76291232638889
	 Validation accuracy  97.87556966145833
	 Epoch Loss  1.7924761772155762
Epoch Number :  190
	 Training accuracy:  99.05232747395833
	 Validation accuracy  97.398681640625
	 Epoch Loss  1.7905749082565308
Epoch Number :  191
	 Training accuracy:  99.10203721788194
	 Validation accuracy  97.68208821614583
	 Epoch Loss  1.7904571294784546
Epoch Number :  192
	 Training accuracy:  98.81401909722223
	 Validation accuracy  97.39176432291667
	 Epoch Loss  1.7897062301635742
Epoch Number :  193
	 Training accuracy:  99.19325086805556
	 Validation accuracy  97.80375162760417
	 Epoch Loss  1.7884539365768433
Epoch Number :  194
	 Training accuracy:  98.9923095703125
	 Validat

Epoch Number :  249
	 Training accuracy:  99.287109375
	 Validation accuracy  97.36958821614583
	 Epoch Loss  1.7207725048065186
Epoch Number :  250
	 Training accuracy:  99.26317003038194
	 Validation accuracy  97.545166015625
	 Epoch Loss  1.7192260026931763
Epoch Number :  251
	 Training accuracy:  99.28738064236111
	 Validation accuracy  97.27254231770833
	 Epoch Loss  1.7180780172348022
Epoch Number :  252
	 Training accuracy:  99.27313910590277
	 Validation accuracy  97.3858642578125
	 Epoch Loss  1.7167909145355225
Epoch Number :  253
	 Training accuracy:  99.25265842013889
	 Validation accuracy  97.24690755208333
	 Epoch Loss  1.7151215076446533
Epoch Number :  254
	 Training accuracy:  99.33810763888889
	 Validation accuracy  97.43082682291667
	 Epoch Loss  1.7150096893310547
Epoch Number :  255
	 Training accuracy:  99.57865397135417
	 Validation accuracy  97.39888509114583
	 Epoch Loss  1.713099479675293
Epoch Number :  256
	 Training accuracy:  99.12929958767361
	 Validatio

In [ ]:
print(train_target.shape)

In [ ]:
print(train_target[44,:,:,:])

#### 

In [ ]:
print(output.shape)

In [ ]:
print(output[14,1,:,:,:])

In [ ]:
training_accuracy = np.array(100*(Ntrain_nb-train_errors)/Ntrain_nb)
validation_accurcy = np.array(100*(Nvalidation_nb-validation_errors)/Nvalidation_nb)

In [ ]:
plt.plot(training_accuracy)
plt.plot(validation_accurcy)
plt.show()

In [ ]:
test_visualisation = output[14,1,:,:,:].round()

In [ ]:
voxels = np.array(test_visualisation.data)

# and plot everything
fig = plt.figure(figsize=(10,10))
ax = fig.gca(projection='3d')
ax.voxels(voxels)
fig.savefig('VoxelizedFinal.png')
fig.show()

In [ ]:
'''def summary(input_size, model):
        def register_hook(module):
            def hook(module, input, output):
                class_name = str(module.__class__).split('.')[-1].split("'")[0]
                module_idx = len(summary)

                m_key = '%s-%i' % (class_name, module_idx+1)
                summary[m_key] = OrderedDict()
                summary[m_key]['input_shape'] = list(input[0].size())
                summary[m_key]['input_shape'][0] = -1
                summary[m_key]['output_shape'] = list(output.size())
                summary[m_key]['output_shape'][0] = -1

                params = 0
                if hasattr(module, 'weight'):
                    params += th.prod(th.LongTensor(list(module.weight.size())))
                    if module.weight.requires_grad:
                        summary[m_key]['trainable'] = True
                    else:
                        summary[m_key]['trainable'] = False
                if hasattr(module, 'bias'):
                    params +=  th.prod(th.LongTensor(list(module.bias.size())))
                summary[m_key]['nb_params'] = params
                
            if not isinstance(module, nn.Sequential) and \
               not isinstance(module, nn.ModuleList) and \
               not (module == model):
                hooks.append(module.register_forward_hook(hook))
                
        dtype = th.cuda.FloatTensor
        
        # check if there are multiple inputs to the network
        if isinstance(input_size[0], (list, tuple)):
            x = [Variable(th.rand(1,*in_size)).type(dtype) for in_size in input_size]
        else:
            x = Variable(th.rand(1,*input_size)).type(dtype)
            
        print(x.shape)
        print(type(x[0]))
        # create properties
        summary = OrderedDict()
        hooks = []
        # register hook
        model.apply(register_hook)
        # make a forward pass
        model(x)
        # remove these hooks
        for h in hooks:
            h.remove()

        print('----------------------------------------------------------------')
        line_new = '{:>20}  {:>25} {:>15}'.format('Layer (type)', 'Output Shpae', 'Param #')
        print(line_new)
        print('================================================================')
        total_params = 0
        trainable_params = 0
        for layer in summary:
            ## input_shape, output_shape, trainable, nb_params
            line_new = '{:>20}  {:>25} {:>15}'.format(layer, summary[layer]['output_shape'], summary[layer]['nb_params'])
            total_params += summary[layer]['nb_params']
            if 'trainable' in summary[layer]:
                if summary[layer]['trainable'] == True:
                    trainable_params += summary[layer]['nb_params']
            print(line_new)
        print('================================================================')
        print('Total params: ' + str(total_params))
        print('Trainable params: ' + str(trainable_params))
        print('Non-trainable params: ' + str(total_params - trainable_params))
        print('----------------------------------------------------------------')
        return summary'''